In [3]:
import pandas as pd
import numpy as np

In [4]:
# read the large csv file with specified chunksize 
# df_chunk = pd.read_csv("//ant/dept-eu/LHR16/Dept4/AIV-TVOD/BI/PMD_Analysis/PMD_data_4.txt", sep='\t', 
#                        low_memory = False, encoding = 'ISO-8859-1', chunksize = 100000)

# # df_chunk = pd.read_csv("//ant/dept-eu/LHR16/Dept4/AIV-TVOD/BI/PMD_Analysis/PMD_data_4.txt", sep='\t', 
# #                        header=None, low_memory = False, encoding = 'ISO-8859-1')

In [5]:
# chunk_list = []  # append each chunk df here 

# # Each chunk is in df format
# for chunk in df_chunk:  
#     # perform data filtering 
#     #chunk_filter = chunk_preprocessing(chunk)
    
#     # Once the data filtering is done, append the chunk to list
#     chunk_list.append(chunk)
    
# # concat the list into dataframe 
# df_concat = pd.concat(chunk_list)

In [6]:
#df_concat.to_pickle("data_granularDE.pkl")

In [7]:
df_concatenated = pd.read_pickle("data_granularDE.pkl")

In [8]:
df_concatenated.head()

,marketplace_id,encrypted_customer_id,unencrypted_customer_id,transaction_date_local,transaction_datetime_local,week_ending,revenue,units,content_age,first_purchase_date,...,is_active_ttm,total_units_ttm,total_revenue_ttm,total_units_before_ttm,total_revenue_before_ttm,cumulative_sum_total,promo_dist,running_arpu_ever,running_tenure_ever,running_arpu_monthly_ever
0,4,A000566631H70KG0OKKF7,9223372034587508951,2019-04-22 00:00:00,2019-04-22 01:32:10,2019-12-07 00:00:00,2.09,1,A_Under_30d,2019-04-22 01:32:10,...,1,9.0,31.4,NaN,NaN,2.09,non_promo,2.0900,0.0,2.0900
1,4,A000566631H70KG0OKKF7,9223372034587508951,2019-04-22 00:00:00,2019-04-22 03:16:54,2019-12-07 00:00:00,3.34,1,D_1y_2yr,2019-04-22 01:32:10,...,1,9.0,31.4,NaN,NaN,5.43,non_promo,2.7150,0.0,5.4300
2,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-15 00:00:00,2019-07-15 02:16:24,2019-12-07 00:00:00,4.19,1,A_Under_30d,2019-04-22 01:32:10,...,1,9.0,31.4,NaN,NaN,9.62,non_promo,3.2066,3.0,3.2066
3,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-23 00:00:00,2019-07-23 04:49:23,2019-12-07 00:00:00,3.35,1,B_30days_6m,2019-04-22 01:32:10,...,1,9.0,31.4,NaN,NaN,12.97,non_promo,3.2425,3.0,4.3233
4,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-28 00:00:00,2019-07-28 01:49:41,2019-12-07 00:00:00,3.35,1,D_1y_2yr,2019-04-22 01:32:10,...,1,9.0,31.4,NaN,NaN,16.32,non_promo,3.2640,3.0,5.4400


In [9]:
# df_concat.columns = ['marketplace_id',"encrypted_customer_id","unencrypted_customer_id","transaction_date_local",
#                     "transaction_datetime_local","week_ending","revenue","units","content_age","first_purchase_date",
#                     "series_or_movie_asin","series_or_movie_name","is_promo","is_pmd","content_type","total_units",
#                      "total_revenue","rank_transactions","is_active_ttm","total_units_ttm","total_revenue_ttm",
#                     "total_units_before_ttm","total_revenue_before_ttm","cumulative_sum_total","promo_dist","running_arpu_ever",
#                     "running_tenure_ever","running_arpu_monthly_ever"]

In [10]:
def make_prep(df):

    df['transaction_date_local'] = pd.to_datetime(df['transaction_date_local'])
    df['transaction_datetime_local'] = pd.to_datetime(df['transaction_datetime_local'])
    df['first_purchase_date'] = pd.to_datetime(df['first_purchase_date'])
    df['tenure'] = [int(i.days) for i in (max(df['transaction_date_local']) - df['first_purchase_date'])]
    df['cumulative_sum_total'] = df['cumulative_sum_total'].astype(float)
    df['tenure_days_up_to_last_purchase'] = [int(i.days) for i in (df['transaction_date_local']) - df['first_purchase_date']] 
    df['tenure_days_up_to_last_purchase_corrected'] = np.where(df['tenure_days_up_to_last_purchase'] < 0, 0, df['tenure_days_up_to_last_purchase'] )
    
    df['tenure_months_up_to_last_purchase'] = df['tenure_days_up_to_last_purchase'] / 30
    df['tenure_up_to_last_purchase_corrected'] = np.where(df['tenure_months_up_to_last_purchase'] < 1, 1, df['tenure_months_up_to_last_purchase']).astype(float)
    df['arpu_monthly_dynamic'] = df['cumulative_sum_total'] / (df['tenure_up_to_last_purchase_corrected'])
    
    df['frequency'] = df.groupby('encrypted_customer_id')['tenure_days_up_to_last_purchase_corrected'].diff()
    
    pd.set_option('mode.chained_assignment', None)
      # set default value
    df['tenure_years'] = "default"
    df['tenure_years'][(df['tenure'] < 360)] = "under_year"
    df['tenure_years'][(df['tenure'] >= 360) & (df['tenure'] < 360*2)] = 'btw_1and2yrs'
    df['tenure_years'][(df['tenure'] >= 360*2) & (df['tenure'] < 360*3)] = 'btw_2and3yrs'
    df['tenure_years'][(df['tenure'] >= 360*3) & (df['tenure'] < 360*4)] = 'btw_3and4yrs'
    df['tenure_years'][(df['tenure'] >= 360*4) & (df['tenure'] < 360*5)] = 'btw_4and5yrs'
    df['tenure_years'][(df['tenure'] >= 360*5) ] = 'over_5years'
    
    
    df = df.fillna(0)
    
    return df

In [11]:
df = make_prep(df_concatenated)

In [12]:
df.head()

,marketplace_id,encrypted_customer_id,unencrypted_customer_id,transaction_date_local,transaction_datetime_local,week_ending,revenue,units,content_age,first_purchase_date,...,running_tenure_ever,running_arpu_monthly_ever,tenure,tenure_days_up_to_last_purchase,tenure_days_up_to_last_purchase_corrected,tenure_months_up_to_last_purchase,tenure_up_to_last_purchase_corrected,arpu_monthly_dynamic,frequency,tenure_years
0,4,A000566631H70KG0OKKF7,9223372034587508951,2019-04-22,2019-04-22 01:32:10,2019-12-07 00:00:00,2.09,1,A_Under_30d,2019-04-22 01:32:10,...,0.0,2.0900,240,-1,0,-0.033333,1.000000,2.090000,0.0,under_year
1,4,A000566631H70KG0OKKF7,9223372034587508951,2019-04-22,2019-04-22 03:16:54,2019-12-07 00:00:00,3.34,1,D_1y_2yr,2019-04-22 01:32:10,...,0.0,5.4300,240,-1,0,-0.033333,1.000000,5.430000,0.0,under_year
2,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-15,2019-07-15 02:16:24,2019-12-07 00:00:00,4.19,1,A_Under_30d,2019-04-22 01:32:10,...,3.0,3.2066,240,83,83,2.766667,2.766667,3.477108,83.0,under_year
3,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-23,2019-07-23 04:49:23,2019-12-07 00:00:00,3.35,1,B_30days_6m,2019-04-22 01:32:10,...,3.0,4.3233,240,91,91,3.033333,3.033333,4.275824,8.0,under_year
4,4,A000566631H70KG0OKKF7,9223372034587508951,2019-07-28,2019-07-28 01:49:41,2019-12-07 00:00:00,3.35,1,D_1y_2yr,2019-04-22 01:32:10,...,3.0,5.4400,240,96,96,3.200000,3.200000,5.100000,5.0,under_year


In [13]:
df.columns

Index(['marketplace_id', 'encrypted_customer_id', 'unencrypted_customer_id',
       'transaction_date_local', 'transaction_datetime_local', 'week_ending',
       'revenue', 'units', 'content_age', 'first_purchase_date',
       'series_or_movie_asin', 'series_or_movie_name', 'is_promo', 'is_pmd',
       'content_type', 'total_units', 'total_revenue', 'rank_transactions',
       'is_active_ttm', 'total_units_ttm', 'total_revenue_ttm',
       'total_units_before_ttm', 'total_revenue_before_ttm',
       'cumulative_sum_total', 'promo_dist', 'running_arpu_ever',
       'running_tenure_ever', 'running_arpu_monthly_ever', 'tenure',
       'tenure_days_up_to_last_purchase',
       'tenure_days_up_to_last_purchase_corrected',
       'tenure_months_up_to_last_purchase',
       'tenure_up_to_last_purchase_corrected', 'arpu_monthly_dynamic',
       'frequency', 'tenure_years'],
      dtype='object')

In [14]:
df['promo_dist'].unique()

array(['non_promo', 'pmd_promo', 'vendisto_promo'], dtype=object)

In [15]:
#Create a dataset with first pmd transaction
subset_first_transaction_by_type = df.groupby(['encrypted_customer_id','promo_dist'])['rank_transactions'].min().reset_index()

In [16]:
subset_first_transaction_by_type.columns = ['encrypted_customer_id','promo_dist_entry','rank_transactions']
subset_first_transaction_by_type.head()

,encrypted_customer_id,promo_dist_entry,rank_transactions
0,A000008620RRI25LDWMJZ,non_promo,1
1,A00009662PVU742JLBPTP,non_promo,1
2,A0001210DHVG1EB1OKP5,non_promo,1
3,A0001294FIJCW4LXHO9Y,non_promo,1
4,A0001558Z8QHBLZA9AA1,non_promo,2


In [17]:
df[df['promo_dist'] == 'pmd_promo']['encrypted_customer_id'].nunique()

362636

In [18]:
# Create a copy of the whole set
copy_dataset = df[["marketplace_id", "encrypted_customer_id","transaction_date_local","revenue","units","content_age",
                   "arpu_monthly_dynamic","frequency","total_units", "total_revenue",
                   "total_units_ttm", "total_revenue_ttm","rank_transactions",
                   "total_units_before_ttm", "total_revenue_before_ttm", "promo_dist",
                   "tenure_years","tenure_up_to_last_purchase_corrected","tenure"]]

In [19]:
print(copy_dataset.shape)

(13550327, 19)


# Analyse different entry points into PMD

In [20]:
def make_final_adjastments(df):
    subset_first_transaction_by_type = df.groupby(['encrypted_customer_id','promo_dist'])['rank_transactions'].min().reset_index()
    subset_first_transaction_by_type.columns = ['encrypted_customer_id','promo_dist_entry','rank_transactions']
    
    copy_dataset = df[["marketplace_id", "encrypted_customer_id","transaction_date_local","revenue","units","content_age",
                   "arpu_monthly_dynamic","frequency","total_units", "total_revenue",
                   "total_units_ttm", "total_revenue_ttm","rank_transactions",
                   "total_units_before_ttm", "total_revenue_before_ttm", "promo_dist",
                   "tenure_years","tenure_up_to_last_purchase_corrected","tenure","is_active_ttm"]]
    df_new = pd.merge(left = copy_dataset, right = subset_first_transaction_by_type, left_on = ['encrypted_customer_id','rank_transactions'],
                 right_on = ['encrypted_customer_id','rank_transactions'], how = 'left')
    
    
    subset_first_transaction_pmd = subset_first_transaction_by_type[subset_first_transaction_by_type['promo_dist_entry'] == 'pmd_promo']
    subset_first_transaction_pmd.columns = ['encrypted_customer_id','promo_dist_entry_specific','rank_transactions_entry']
    
    df_new = pd.merge(left = df_new, right = subset_first_transaction_pmd[['encrypted_customer_id','rank_transactions_entry']],
                      left_on = ['encrypted_customer_id'],
                      right_on = ['encrypted_customer_id'], how = 'left')
    
    df_new['rank_transactions_entry'] = df_new['rank_transactions_entry'].fillna(0)
    
    
    pd.set_option('mode.chained_assignment', None)
    
    df_new['which_relative_to_pmd'] = 'no_entry'
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] < df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0) ] = "before"
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] > df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0)] = "after"
    df_new['which_relative_to_pmd'][(df_new['rank_transactions'] == df_new['rank_transactions_entry']) &
                                   (df_new['rank_transactions_entry'] != 0)] = "pmd_entry"
    
    return df_new

In [21]:
df_new = make_final_adjastments(df)

In [22]:
df_new.shape

(13550327, 23)

In [23]:
df_new.columns

Index(['marketplace_id', 'encrypted_customer_id', 'transaction_date_local',
       'revenue', 'units', 'content_age', 'arpu_monthly_dynamic', 'frequency',
       'total_units', 'total_revenue', 'total_units_ttm', 'total_revenue_ttm',
       'rank_transactions', 'total_units_before_ttm',
       'total_revenue_before_ttm', 'promo_dist', 'tenure_years',
       'tenure_up_to_last_purchase_corrected', 'tenure', 'is_active_ttm',
       'promo_dist_entry', 'rank_transactions_entry', 'which_relative_to_pmd'],
      dtype='object')

In [24]:
ch = df_new[df_new['encrypted_customer_id'].isin(['A10960PTH4K18U','A000566631H70KG0OKKF7','A1033MZXPVO47W'])]

In [274]:
#ch[ch['encrypted_customer_id'] == 'A1033MZXPVO47W'].iloc[:,3:]
#ch[ch['encrypted_customer_id'] == 'A10960PTH4K18U'].iloc[:,3:]
#ch

In [26]:
df_new['which_relative_to_pmd'].unique()

array(['no_entry', 'pmd_entry', 'after', 'before'], dtype=object)

In [27]:
def get_diff_stats(x):
    """provides various metrics for count, sum, revenue"""
    d = {}
    d['count_customers'] = x['encrypted_customer_id'].nunique()
    d['arpu_monthly_dynamic'] = x['arpu_monthly_dynamic'].mean()
    d['average_frequency'] = x['frequency'].mean()
    d['units'] = x['units'].sum()
    d['revenue'] = x['revenue'].sum()
    
    return pd.Series(d, index = ['count_customers','arpu_monthly_dynamic', 'average_frequency', 'units','revenue'])

# Subset of those who have done BOTH before and after

In [230]:
def create_subset(df):
    before = df[df['which_relative_to_pmd'].isin(['before'])]['encrypted_customer_id'].unique()
    after_with_before = df[(df['which_relative_to_pmd'] == 'after') &
                           df['encrypted_customer_id'].isin(before)]['encrypted_customer_id'].unique()
    
    subset = df[df['encrypted_customer_id'].isin(after_with_before)]
    
    return subset

In [235]:
d = create_subset(df_new)

In [237]:
d.shape

(3620616, 23)

In [238]:
grouped = d.groupby(["encrypted_customer_id", "total_units","tenure_years",
                       "total_units_ttm","rank_transactions_entry","which_relative_to_pmd"]).apply(get_diff_stats).reset_index()

In [240]:
grouped.head()

,encrypted_customer_id,total_units,tenure_years,total_units_ttm,rank_transactions_entry,which_relative_to_pmd,count_customers,arpu_monthly_dynamic,average_frequency,units,revenue
0,A000273015K0TYSUARO87,23,btw_3and4yrs,6.0,12.0,after,1.0,2.040812,58.454545,11.0,36.86
1,A000273015K0TYSUARO87,23,btw_3and4yrs,6.0,12.0,before,1.0,2.413213,39.454545,11.0,40.21
2,A000273015K0TYSUARO87,23,btw_3and4yrs,6.0,12.0,pmd_entry,1.0,2.597468,40.000000,1.0,0.83
3,A000698623JMNYEX0WKEA,10,btw_3and4yrs,0.0,2.0,after,1.0,1.147479,81.500000,8.0,32.25
4,A000698623JMNYEX0WKEA,10,btw_3and4yrs,0.0,2.0,before,1.0,4.190000,0.000000,1.0,4.19


In [247]:
print(grouped['encrypted_customer_id'].nunique())
print(grouped[grouped['which_relative_to_pmd'] == 'before']['encrypted_customer_id'].nunique())
print(grouped[grouped['which_relative_to_pmd'] == 'after']['encrypted_customer_id'].nunique())

245159
245159
245159


In [248]:
grouped_no_cust = d.groupby(["total_units","tenure_years",
                       "total_units_ttm","rank_transactions_entry","which_relative_to_pmd"]).apply(get_diff_stats).reset_index()

In [251]:
print(grouped_no_cust.shape)
grouped_no_cust.head()

(90423, 10)


,total_units,tenure_years,total_units_ttm,rank_transactions_entry,which_relative_to_pmd,count_customers,arpu_monthly_dynamic,average_frequency,units,revenue
0,3,btw_1and2yrs,0.0,2.0,after,382.0,4.187200,40.743455,382.0,1283.60
1,3,btw_1and2yrs,0.0,2.0,before,382.0,4.855157,0.000000,382.0,1854.67
2,3,btw_1and2yrs,0.0,2.0,pmd_entry,382.0,3.991889,75.201571,382.0,600.06
3,3,btw_1and2yrs,1.0,2.0,after,486.0,1.126185,273.954733,486.0,2246.26
4,3,btw_1and2yrs,1.0,2.0,before,486.0,4.779733,0.000000,486.0,2322.95


In [253]:
#grouped_no_cust.to_csv("grouped_cleanDE.csv")

In [254]:
print(d['encrypted_customer_id'].nunique())
print(d[d['which_relative_to_pmd'] == 'before']['encrypted_customer_id'].nunique())
print(d[d['which_relative_to_pmd'] == 'after']['encrypted_customer_id'].nunique())

245159
245159
245159


In [260]:
ch = d[d['encrypted_customer_id'].isin(['A1033MZXPVO47W','AZWOM6LA5WIKD'])]

In [267]:
grouped_by_dist = d.groupby(["total_units","tenure_years",
                       "total_units_ttm","rank_transactions_entry","which_relative_to_pmd","promo_dist"]).apply(get_diff_stats).reset_index()

In [272]:
grouped_by_dist.shape

(129613, 11)

In [273]:
grouped_by_dist.to_csv("grouped_by_distDE.csv")